# OpenPTV calibration using given markers and Scipy

In [51]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize

from openptv_python.calibration import Calibration
from openptv_python.parameters import ControlPar, VolumePar, OrientPar

from openptv_python.imgcoord import image_coordinates
from openptv_python.trafo import arr_metric_to_pixel
from openptv_python.orientation import external_calibration, full_calibration


import pathlib, os
working_path = pathlib.Path.cwd()

In [52]:
class Parameter:
    cams = [0,1,2,3]
    Vmin = [0,0,0]
    Vmax = [300,300,300]
    N1, N2 = 361, 5

In [53]:
params = Parameter()

markers = [np.loadtxt('../proPTV_LineOfSight/markers_c'+str(cam)+'.txt') for cam in params.cams]
XYZ = markers[0][:,2:]
xy = markers[0][:,:2]
ID = np.argwhere((XYZ[:,0]>-1))[:,0]

In [54]:
xyz = pd.DataFrame(XYZ, columns=['x','y','z'])
xyz['id'] = ID
# px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()

In [55]:
# First, let's calibrate roughly the cameras

In [56]:

ref_pts = XYZ[[0,721,1409,1462],:]
xyz = pd.DataFrame(ref_pts, columns=['x','y','z'])
xyz['id'] = ID[[0,721,1409,1462]]
# px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()


In [57]:
cam_id = 1 # or 2,3,4

In [65]:
cal = Calibration().from_file(working_path / "calibration" / f"cam{cam_id}.tif.ori", None)
cpar = ControlPar().from_file(working_path / "parameters" / "ptv.par")
vpar = VolumePar().from_file(working_path / "parameters" / "criteria.par")

No addpar file found. Using default values for radial distortion


In [66]:
four_points = xy[[0,721,1409,1462],:]
print(f"{four_points = }")

four_points = array([[ 315., 1996.],
       [2133.,  235.],
       [ 892.,  561.],
       [2215., 2018.]])


In [67]:
external_calibration(cal, ref_pts, four_points, cpar)
print(f"{cal.ext_par = }")

cal.ext_par = rec.array((358.83959235, -13.80970931, 1144.65026509, 0.1774831, 0.2116152, -0.03131282, [[ 0.97721366,  0.03060932,  0.21003934], [ 0.00624895,  0.98496965, -0.1726144 ], [-0.21216599,  0.16999368,  0.96233453]]),
          dtype=[('x0', '<f8'), ('y0', '<f8'), ('z0', '<f8'), ('omega', '<f8'), ('phi', '<f8'), ('kappa', '<f8'), ('dm', '<f8', (3, 3))])


In [68]:
targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, cpar.mm),
cpar,
)
four_points - targets

array([[-4.70252132,  4.72348146],
       [ 6.1647693 , -5.90960052],
       [ 6.35980315,  8.8123377 ],
       [-7.56655596, -7.37669387]])

In [69]:
def array_to_calibration(x:np.ndarray, cal:Calibration) -> None:
    # cal = Calibration()
    cal.set_pos(x[:3])
    cal.set_angles(x[3:6])
    cal.set_primary_point(x[6:9])
    # cal.set_radial_distortion(x[9:12])
    # cal.set_decentering(x[12:14])
    # cal.set_affine_distortion(x[14:])
    return None

def calibration_to_array(cal:Calibration) -> np.ndarray:
    return np.concatenate([
        cal.get_pos(),
        cal.get_angles(),
        cal.get_primary_point(),
        cal.get_radial_distortion(),
        cal.get_decentering(),
        cal.get_affine(),
    ])

In [72]:
def error_function(x, cal, XYZ, xy, cpar):
    
    array_to_calibration(x, cal)
    
    targets = arr_metric_to_pixel(
        image_coordinates(XYZ, cal, cpar.mm),
    cpar,
    )
    # err = np.sum(np.abs(xy - targets))
    err = np.sum(xy - targets)
    # print(err)
    return err

In [73]:

x0 = calibration_to_array(cal)
print(x0)
sol = minimize(error_function, x0, args=(cal, XYZ, xy, cpar), method='Nelder-Mead', tol=1e-4)

In [ ]:
array_to_calibration(sol.x, cal)

targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, cpar.mm),
cpar,
)
four_points - targets


array([[ 1.4940228 , -3.39689642],
       [-3.20540632,  1.24652253],
       [ 0.70491151,  0.07242609],
       [-1.6004268 ,  0.17023819]])

In [ ]:
targets = arr_metric_to_pixel(
    image_coordinates(XYZ, cal, cpar.mm),
cpar,
)


In [ ]:

import plotly.figure_factory as ff
# px.scatter(x=xy[:,0], y=xy[:,1], color=ID).show()
fig = ff.create_quiver(x=xy[:,0], y=xy[:,1], u=targets[:,0]-xy[:,0], v=targets[:,1]-xy[:,1], scale=5)
fig.show()


In [ ]:
cal.write(working_path / "calibration" / "cam{cam_id}_scipy.ori", working_path / "calibration" / "cam{cam_id}_scipy.addpar")